In [1]:
#import libraries to make wind rose diagram
import geopandas as gpd
import os
import numpy as np
import pandas as pd
import pvlib
from pvlib.location import Location


In [2]:
# Set paths
wd = r'S:\Users\stidjaco\R_files\BigPanel'
derived_path = os.path.join(wd, r'Data/Derived')
derivedTemp_path = os.path.join(derived_path, r'intermediateProducts')
gmseusArraysPath = os.path.join(derivedTemp_path, r'GMSEUS_Arrays_estGCR.shp')

# Call gmseusArrays
gmseusArrays = gpd.read_file(gmseusArraysPath)

# If mount string contains 'fixed','mixed_fs', 'mixed_df', 'mixed_dfs', or 'mixed', maintain tilt, otherwise set tilt to -9999. These come from checked arrays (errors in permitting data?)
gmseusArrays['tilt'] = gmseusArrays.apply(lambda row: row['tilt'] if any(x in str(row['mount']).lower() for x in ['fixed_axis', 'mixed_fs', 'mixed_df', 'mixed_dfs', 'mixed']) else -9999, axis=1)

# Assign array latitude and longitude based on the centroid of the array geometry (using NAD83 -- EPSG:4269)
gmseusArrays4269 = gmseusArrays.copy()
gmseusArrays4269 = gmseusArrays4269.to_crs(epsg=4269)
gmseusArrays['latitude'] = gmseusArrays4269['geometry'].centroid.y
gmseusArrays['longitude'] = gmseusArrays4269['geometry'].centroid.x

# GM-SEUS Arrays (gmseusArrays) contains the following attributes for estimating tilt: 'avgAimuth' (degrees from north), 'GCR1' (rowArea / totArea), and 'mount' (e.g., 'fixed_axis'). 
# We have also just added latitude and longitude here to gmseusArrays for the tilt estimation.
# We have also aready checked existing array infomation for tilt, so we will call estimated tilt a new column:'tiltEst'.
# Mounts with tilt information include: 'fixed_axis','mixed_fs', 'mixed_df', 'mixed_dfs', or 'mixed'. 

# Get gmseusArrays with mounts containing: 'fixed_axis','mixed_fs', 'mixed_df', 'mixed_dfs', or 'mixed'
gmseusArrays_estTilt = gmseusArrays[gmseusArrays['mount'].str.contains('fixed_axis|mixed_fs|mixed_df|mixed_dfs|mixed', case=False)].reset_index(drop=True)


C:\Users\bingam15\AppData\Local\Temp\ipykernel_32940\2594564008.py:16: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gmseusArrays['latitude'] = gmseusArrays4269['geometry'].centroid.y
C:\Users\bingam15\AppData\Local\Temp\ipykernel_32940\2594564008.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gmseusArrays['longitude'] = gmseusArrays4269['geometry'].centroid.x


In [4]:

# For each  row in the gmseusArrays DataFrame, find the angle of tilt that maximizes annual global POA irradiance for a local TMY
# Uses the latitude and longitude of the location to get the TMY data
# The azimuth of the array is provided in the DataFrame as avgAzimuth

gmseusArrays_estTilt = gmseusArrays_estTilt.head(50)

#subset the gmseusArrays DataFrame to the first 50 rows
# Loop through the rows of the gmseusArrays DataFrame to perform the tilt estimation for each array
for index, row in gmseusArrays_estTilt.iterrows():


    # Define the latitude and longitude of the  array location
    latitude = row['latitude']
    longitude = row['longitude']


    # Use the pvgis tool to get location specific TMY data
    # Global TMY data from pvgis
    tmy_data = pvlib.iotools.get_pvgis_tmy(latitude, longitude, outputformat='basic')
    
    # basic output returns a tuple that is 4 elements long. The first element is the TMY DataFrame
    tmy_data = tmy_data[0]
    
    # Ensure the index is a DatetimeIndex
    tmy_data.index = pd.to_datetime(tmy_data.index)

    # define the location and use it to get solar position data that corresponds to the TMY data
    location = pvlib.location.Location(latitude, longitude)
    solar_position = location.get_solarposition(tmy_data.index)

    #create a data frame for the results of panel tilet and total annual global POA irradiance
    results = pd.DataFrame(columns=['array_tilt', 'total_global_poa'])

    # Loop through the possible array tilts from 0 to 90 degrees and calculate the total annual global POA irradiance
    for array_tilt in range(0, 90, 1):
        poa_irradiance = pvlib.irradiance.get_total_irradiance(
        surface_tilt=array_tilt,
        surface_azimuth=row['avgAzimuth'],
        dni=tmy_data['dni'],
        ghi=tmy_data['ghi'],
        dhi=tmy_data['dhi'],
        solar_zenith=solar_position['zenith'],
        solar_azimuth=solar_position['azimuth'],)

        # Sum the Global POA Irradiance to get the total annual global POA irradiance
        total_global_poa = poa_irradiance['poa_global'].sum()
        
        # Add the results to the results DataFrame
        new_row = pd.DataFrame({'array_tilt': [array_tilt], 'total_global_poa': [total_global_poa]})
        results = pd.concat([results, new_row], ignore_index=True)
        
    # find the maximum total annual energy and the corresponding array tilt
    max_poa = results['total_global_poa'].max()
    est_tilt = results.loc[results['total_global_poa'] == max_poa, 'array_tilt'].values[0]

    # Assign the best tilt to the gmseusArrays DataFrame under the column estTilt
    gmseusArrays_estTilt.loc[index, 'estTilt'] = est_tilt

# Display the DataFrame
print(gmseusArrays_estTilt)

C:\Users\bingam15\AppData\Local\Temp\ipykernel_32940\1462849506.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)
C:\Users\bingam15\AppData\Local\Temp\ipykernel_32940\1462849506.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)
C:\Users\bingam15\AppData\Local\Temp\ipykernel_32940\1462849506.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA e

   nativeID Source  instYr   capMW          area modType AVtype  azimuth  \
0       770  CCVPV  2016.0 -9999.0   2052.954815    c-si   None  -9999.0   
1       674  CCVPV  2013.0 -9999.0   9121.142792    c-si   None  -9999.0   
2        18  CCVPV  2014.0 -9999.0   2919.310707    c-si   None  -9999.0   
3       179  CCVPV  2017.0 -9999.0   3224.351053    c-si   None  -9999.0   
4       417  CCVPV  2016.0 -9999.0   4086.217685    c-si   None  -9999.0   
5       676  CCVPV  2013.0 -9999.0    938.221528    c-si   None  -9999.0   
6       178  CCVPV  2018.0 -9999.0   5547.191586    c-si   None  -9999.0   
7       359  CCVPV  2011.0 -9999.0   6261.148849    c-si   None  -9999.0   
8       177  CCVPV  2016.0 -9999.0   2675.799444    c-si   None  -9999.0   
9       857  CCVPV  2014.0 -9999.0   1754.141299    c-si   None  -9999.0   
10      180  CCVPV  2017.0 -9999.0   4955.564477    c-si   None  -9999.0   
11      176  CCVPV  2015.0 -9999.0   4616.446121    c-si   None  -9999.0   
12      175 

C:\Users\bingam15\AppData\Local\Temp\ipykernel_32940\1462849506.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)
